In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append('../RegSeq/')
import utils

ModuleNotFoundError: No module named 'pymc'

In [13]:
%matplotlib inline

# logomaker import
import logomaker

In [14]:
#We will first load in a binding site matrix.
arraydf = pd.io.parsers.read_csv('../data/example_arrays/aphAAnaerodataset_alldone_with_largeMCMC194_activator',
                                 delim_whitespace=True,index_col='pos')
#We will rename columns to be useable by the logomaker package
arraydf = arraydf.rename(columns={'val_A':'A','val_C':'C','val_G':'G','val_T':'T'})

We first need to find the proper scaling factor for conversion to information logo. Empirically, it is known
that binding sites have approximately 1 bit of information per base pair.

In [ ]:
#finding scaling factor
target_info = len(arraydf.index)
beta = utils.get_beta_for_effect_df(arraydf,target_info)

#we will now use logomaker to convert our energy matrix to an information matrix for plotting
binding_info = logomaker.transform_matrix(df=beta*arraydf,from_type='weight',to_type='information')

In [ ]:
binding_logo = logomaker.Logo(binding_info,
                         font_name='Stencil Std',
                         vpad=.1,
                         width=.8)

# style using Logo methods
binding_logo.style_spines(visible=False)
binding_logo.style_spines(spines=['left', 'bottom'], visible=True)
binding_logo.style_xticks(rotation=90, fmt='%d', anchor=0)

# style using Axes methods
binding_logo.ax.set_ylabel("Information (bits)", labelpad=-1)
binding_logo.ax.xaxis.set_ticks_position('none')
binding_logo.ax.xaxis.set_tick_params(pad=-1)